In [1]:
# importing the libraries
import pandas as pd
import numpy as np
import os
import glob
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow. keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from pathlib import Path


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Neural_Computing/test_data/ISIC2018_Task3_Test_GroundTruth.csv')

In [3]:
print(data.head(5))

         lesion_id      image_id   dx    dx_type   age     sex localization  \
0  HAMTEST_0000000  ISIC_0034524   nv  follow_up  40.0  female         back   
1  HAMTEST_0000001  ISIC_0034525   nv      histo  70.0    male      abdomen   
2  HAMTEST_0000002  ISIC_0034526  bkl      histo  70.0    male         back   
3  HAMTEST_0000003  ISIC_0034527   nv      histo  35.0    male        trunk   
4  HAMTEST_0000004  ISIC_0034528   nv  follow_up  75.0  female        trunk   

         dataset  
0  vidir_molemax  
1      rosendahl  
2      rosendahl  
3    vienna_dias  
4  vidir_molemax  


In [4]:
data_dir = "/content/drive/MyDrive/Neural_Computing/test_data/ISIC2018_Task3_Test_Images/"
  # set the list of age categories available in the folder
# categories = ["dataset 1","dataset 2"]
  # analyze the images
# for item_ in categories:
  # assigning the location of the images to a variable , 6 classes
# path = os.path.join(data_dir, item_)  

# iterate over each images, considering all the age groups
for img in os.listdir(data_dir):  
  # read one by one image
    # print(path+"/"+img)
    # collecting name of the image 
    image_location = data_dir+"/"+img
    # collect the image array
    img_array = cv2.imread(os.path.join(data_dir, img), cv2.IMREAD_GRAYSCALE)  # convert to array
    print('.............',img_array)
    data['path'] = data['image_id'].map(image_location.format)
    
    # data['image_array'] = data['image_id'].map(img_array)
    # input()
    # plot the image
    # plt.imshow(img_array, cmap='gray')  
    # plt.show()  # displays the last image!
    break  # executing one at a time
# checking the array shape
print(img_array.shape) #(128, 128)
print(data.head(5))

............. [[156 157 157 ... 160 158 159]
 [155 156 157 ... 160 158 160]
 [157 157 156 ... 160 159 159]
 ...
 [154 155 154 ... 142 141 141]
 [155 154 154 ... 141 140 142]
 [155 155 154 ... 142 142 141]]
(450, 600)
         lesion_id      image_id   dx    dx_type   age     sex localization  \
0  HAMTEST_0000000  ISIC_0034524   nv  follow_up  40.0  female         back   
1  HAMTEST_0000001  ISIC_0034525   nv      histo  70.0    male      abdomen   
2  HAMTEST_0000002  ISIC_0034526  bkl      histo  70.0    male         back   
3  HAMTEST_0000003  ISIC_0034527   nv      histo  35.0    male        trunk   
4  HAMTEST_0000004  ISIC_0034528   nv  follow_up  75.0  female        trunk   

         dataset                                               path  
0  vidir_molemax  /content/drive/MyDrive/Neural_Computing/test_d...  
1      rosendahl  /content/drive/MyDrive/Neural_Computing/test_d...  
2      rosendahl  /content/drive/MyDrive/Neural_Computing/test_d...  
3    vienna_dias  /content/d

In [5]:
print(data.columns)

Index(['lesion_id', 'image_id', 'dx', 'dx_type', 'age', 'sex', 'localization',
       'dataset', 'path'],
      dtype='object')


In [6]:
skin_disease_type = {
    'akiec':'Actinic keratoses and intraepithelial carcinoma',
    'bcc': 'Basal cell carcinoma',
    'bkl': 'Benign keratosis-like lesions',
    'df': 'Dermatofibroma',
    'mel': 'Melanoma',
    'nv': 'Melanocytic nevi',
    'akiec': 'Actinic keratoses and intraepithelial carcinoma ',
    'vasc': 'Vascular lesions',
}

data['disease_type'] = data['dx'].map(skin_disease_type.get)
data['disease_type_id'] = pd.Categorical(data['disease_type']).codes

In [7]:
data.head(6)

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,path,disease_type,disease_type_id
0,HAMTEST_0000000,ISIC_0034524,nv,follow_up,40.0,female,back,vidir_molemax,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
1,HAMTEST_0000001,ISIC_0034525,nv,histo,70.0,male,abdomen,rosendahl,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
2,HAMTEST_0000002,ISIC_0034526,bkl,histo,70.0,male,back,rosendahl,/content/drive/MyDrive/Neural_Computing/test_d...,Benign keratosis-like lesions,2
3,HAMTEST_0000003,ISIC_0034527,nv,histo,35.0,male,trunk,vienna_dias,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
4,HAMTEST_0000004,ISIC_0034528,nv,follow_up,75.0,female,trunk,vidir_molemax,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
5,HAMTEST_0000005,ISIC_0034529,mel,histo,NaN,unknown,unknown,external,/content/drive/MyDrive/Neural_Computing/test_d...,Melanoma,5


In [8]:
# def preprocess_train_data():
# assigning the path and list of age categories
# set the path of the folder where the images stored
data_dir = "/content/drive/MyDrive/Neural_Computing/test_data/ISIC2018_Task3_Test_Images/"
# set the list of age categories available in the folder
# categories = ["dataset 1","dataset 2"]
# categories = ["dataset 1"]
# set an image size
IMG_SIZE = 64
#initialize new array to keep all the training data
testing_data = []

# function to read all the images and extract the training images and corresponding classes
def generate_testing_data():
  # iterate through all the age folder and through each image
    # for item_ in categories:  
    #   # access the location to individual images based on the age group
    #     path = os.path.join(data_dir,item_) 
    #     # get the list of available classes based each age group 
    #     class_num = categories.index(item_)  
          # iterate over each image per age folder
  for img in (os.listdir(data_dir)):  
      try:
          # read one by one images and convert it into the grayscale format, which is an array
          img_array = cv2.imread(os.path.join(data_dir,img) ,cv2.IMREAD_GRAYSCALE)  
          # resize the image; means it helps to normalize data size
          new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  
          # adding the extracted trained data and corresponding classes to a list
          testing_data.append([new_array, img])  
      # skip the errors, if there is any possibility to occur
      except Exception as e:  
          pass


generate_testing_data()
#checking the total size of the training data
print(len(testing_data))
# print(training_data)

#initialize an empty list to add the list of training features
feature = []
#initialize an empty list to add the list of training targets
image_name = []

# iterating through the set of training features and targets
for features,label in testing_data:
    # extracted feature from the training_data will be added to the 'feature' variable
    feature.append(features)
    # extracted target from the training_data will be added to the 'target' variable
    image_name.append(label)

# print(feature)
# print(image_name)

1511


In [9]:
labels = pd.DataFrame(image_name)
labels.columns= ['image_id']
labels['image_id'] = labels['image_id'].str.replace('.jpg', '', regex=False)
print(labels)

          image_id
0     ISIC_0035046
1     ISIC_0035055
2     ISIC_0035045
3     ISIC_0035043
4     ISIC_0035056
...            ...
1506  ISIC_0035041
1507  ISIC_0035032
1508  ISIC_0035034
1509  ISIC_0035054
1510  ISIC_0035049

[1511 rows x 1 columns]


In [10]:
required_data = pd.merge(labels,data, left_on='image_id', right_on='image_id', how='left')
required_data

,image_id,lesion_id,dx,dx_type,age,sex,localization,dataset,path,disease_type,disease_type_id
0,ISIC_0035046,HAMTEST_0000477,nv,histo,70.0,female,upper extremity,vidir_modern,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
1,ISIC_0035055,HAMTEST_0000485,nv,consensus,NaN,unknown,unknown,external,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
2,ISIC_0035045,HAMTEST_0000476,nv,follow_up,35.0,female,abdomen,vidir_molemax,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
3,ISIC_0035043,HAMTEST_0000474,bkl,consensus,75.0,male,lower extremity,vidir_molemax,/content/drive/MyDrive/Neural_Computing/test_d...,Benign keratosis-like lesions,2
4,ISIC_0035056,HAMTEST_0000486,nv,follow_up,45.0,female,abdomen,vidir_molemax,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
...,...,...,...,...,...,...,...,...,...,...,...
1506,ISIC_0035041,HAMTEST_0000472,bcc,histo,75.0,male,upper extremity,rosendahl,/content/drive/MyDrive/Neural_Computing/test_d...,Basal cell carcinoma,1
1507,ISIC_0035032,HAMTEST_0000466,bkl,histo,65.0,male,back,rosendahl,/content/drive/MyDrive/Neural_Computing/test_d...,Benign keratosis-like lesions,2
1508,ISIC_0035034,HAMTEST_0000468,nv,follow_up,35.0,male,trunk,vidir_molemax,/content/drive/MyDrive/Neural_Computing/test_d...,Melanocytic nevi,4
1509,ISIC_0035054,HAMTEST_0000484,df,consensus,NaN,unknown,unknown,external,/content/drive/MyDrive/Neural_Computing/test_d...,Dermatofibroma,3


In [11]:
# Test the data using saved model

In [12]:
import cv2

In [13]:
# Function to read and resize images
def read_and_resize_image(image_path, size):
    img = cv2.imread(image_path)
    img = cv2.resize(img, size)
    return img

In [14]:
image_size = (64, 64)

In [15]:
# Apply the function to each image path in the 'path' column of the dataframe
required_data['image'] = required_data['path'].apply(lambda x: read_and_resize_image(x, image_size))

In [22]:
required_data['image'].map(lambda x: x.shape).value_counts()

(64, 64, 3)    1511
Name: image, dtype: int64

In [29]:
# Extract the image data and target labels for test sets

X_test = np.stack(required_data['image'].values)
X_test = X_test/255
y_test = required_data['disease_type_id'].values
y_test = to_categorical(y_test)

In [25]:
# # Create a new DataFrame with 'lesion_id' and 'cell_type_dx' columns from the test data
# result_df = pd.DataFrame({'lesion_id': required_data['lesion_id'], 'target': y_test})
# result_df
# # Save the DataFrame to a CSV file
# result_df.to_csv('submission.csv', index=False)

Done

In [31]:
# load the saved model
from tensorflow import keras
CNN_model = keras.models.load_model('/content/model_v2_.h5')
print('Testing Started \n\n\n')

# CNN_model.evaluate(X_test, Y_test, verbose=0)
predictions_prob = CNN_model.predict(X_test)
predictions = predictions_prob.argmax(axis=-1)
    
# print('Exact prediction',accuracy_cnt)
score = CNN_model.evaluate(X_test, y_test, verbose=0)
# print("Test Score: ", score[0])
print('Test loss:', score[0])
print("Accuracy: ", score[1]*100)


y_test = np.argmax(y_test, axis=-1)
predictions_prob = np.argmax(predictions_prob, axis=-1)
print('\n\n')

print('******classification report******')
print(classification_report(y_test, predictions_prob))
print('__________________________')
# displays the confusion matrix


print('\nConfusion Matrix')
print('__________________________')
y_actual = pd.Series(y_test, name='Actual')
y_predicted = pd.Series(predictions_prob, name='Predicted')
plot_confusion_matrix = pd.crosstab(y_actual, y_predicted)
print(plot_confusion_matrix)

Testing Started 



48/48 [==============================] - 3s 62ms/step
Test loss: 1.3131864070892334
Accuracy:  60.09265184402466



******classification report******
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        43
           1       0.00      0.00      0.00        93
           2       0.00      0.00      0.00       217
           3       0.00      0.00      0.00        44
           4       0.60      1.00      0.75       908
           5       0.00      0.00      0.00       171
           6       0.00      0.00      0.00        35

    accuracy                           0.60      1511
   macro avg       0.09      0.14      0.11      1511
weighted avg       0.36      0.60      0.45      1511

__________________________

Confusion Matrix
__________________________
Predicted    4
Actual        
0           43
1           93
2          217
3           44
4          908
5          171
6           35


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
